# MMseq2 - preparation / matrix making / exploration of raw output files for mmseq2
### most of functions are moved to .py scripts to avoid kernel death
* also have analysis of rare biomes
* also have analysis of any samples with a poor CDS count

In [2]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import datetime
import seaborn as sns
from sklearn import decomposition
import plotly
import altair as alt
from matplotlib.patches import Patch

# mmseq2 output Data processing

input file with the gene and assigned cluster ; python file pulls the sensor from gene

In [7]:
pd.read_csv('/global/cfs/cdirs/kbase/KE-Catboost/HK/mode_0/mode_0_clustered_100_for_df.csv',nrows=5)

,Unnamed: 0,bar,gene,is_cluster
0,3,75512.0,Ga0453188_0000018_9739_11091,True
1,4,75512.0,Ga0453188_0000018_9739_11091,True
2,5,65133.0,Ga0453188_0000024_43626_45344,True
3,6,65133.0,Ga0453188_0000024_43626_45344,True
4,7,85135.0,Ga0453188_0000024_43626_45344,True


In [ ]:
x['is_cluster'] = "True"
x.columns = ['bar', 'gene', 'is_cluster']
x.to_csv("/global/cfs/cdirs/kbase/KE-Catboost/HK/mode_0/mode_0_clustered_11k_for_df.csv")

## 1) Use 'make_matrix.py files in order to pull clusters into catboost matrix
* ~500 files will be processed in 3 hours, so broke into ~10 .py scrips to speed the process

In [6]:
#example output
pd.read_csv("/global/cfs/cdirs/kbase/KE-Catboost/HK/mode_0_count/copy_matrix_ready_28.csv",nrows=5)

,Unnamed: 0,GOLD Analysis Project ID,Ecosystem,Ecosystem Category,Ecosystem Subtype,Ecosystem Type,Specific Ecosystem,index,406.0,886.0,...,89652.0,94600.0,55402.0,58830.0,75303.0,107579.0,1094.0,94252.0,4964.0,92308.0
0,0,Ga0334848,Environmental,Terrestrial,Wetlands,Soil,Peat,gene_copy,7.0,20.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,Ga0334848,Environmental,Terrestrial,Wetlands,Soil,Peat,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,Ga0334884,Engineered,Bioreactor,Unclassified,Anaerobic,Unclassified,gene_copy,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,Ga0334884,Engineered,Bioreactor,Unclassified,Anaerobic,Unclassified,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,Ga0325404,Host-associated,Plants,Stem,Phyllosphere,Wood/Secondary xylem,gene_copy,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 2) Use the 'condence_matrix.py' to merge files, add the biome label, remove all 0's
## 3) Divide by the total gene_copy_number protein count in all files for abundance

# Mmseq2 data prep
* Use the below to read in amino acids in order to do mmseq2
* Ended up breaking into different biomes, so that it would not crash the kernel

In [ ]:
main_dir = "/global/cfs/cdirs/kbase/jungbluth/HK_sensors/run2/5000_2/5000_1/"
parent_list = os.listdir(main_dir)
my_columns = pd.Series(['GOLD Analysis Project ID', 'gene_oid','pfam_id', 'gene_copy', 'Ecosystem', 'Ecosystem Category',
                        'Ecosystem Subtype', 'Ecosystem Type', 'Specific Ecosystem', 'query_start', 'query_end',
                       'subj_start', 'subj_end', 'bit_score', 'faa_sequence'], )
all_studies_aa = pd.DataFrame()
count = 0
for child in parent_list:
    #if count < 3000: #20,000 total
    if 'pfam-hits_table.tsv' in child:
            # read in file and extract the biome info
        metagenome = pd.read_csv(os.path.join(main_dir,child), delimiter="\t", usecols=my_columns)
        if metagenome.Ecosystem[0] == "Environmental":
            # remove non-sensory, non-conserved domains
            metagenome = metagenome[~metagenome.pfam_id.str.contains("pfam02895|pfam18947|pfam00672|pfam01627|pfam07536|pfam00072")]
            # filter to just conserved domains - has to contain both here
            just_HK_conserved = metagenome[metagenome.pfam_id.str.contains("pfam00512|pfam02518")]
            just_HK_conserved = just_HK_conserved.groupby('gene_oid').count().reset_index()
            just_HK_conserved = just_HK_conserved[['gene_oid']].drop_duplicates()
            just_HK_conserved["has_hk"] = True
            metagenome = metagenome.merge(just_HK_conserved, how='left', on='gene_oid')
            metagenome = metagenome[metagenome.has_hk == True]
            metagenome = metagenome[~metagenome.pfam_id.str.contains("pfam00512|pfam02518")]
            
            all_studies_aa = pd.concat([all_studies_aa, metagenome])
    #else:
       # break
    if count%500 == 1:
        print(count)
        print(datetime.datetime.now())
    count = count+1

all_studies_aa = all_studies_aa.fillna(0)

all_studies_aa.to_csv("/global/cfs/cdirs/kbase/KE-Catboost/HK/pfam_amino_acid_all_study_engineered_1.csv", index='False')

In [ ]:
df = pd.read_csv("/global/cfs/cdirs/kbase/KE-Catboost/HK/matrix_copy_number/catboost_matrix_copy_number.csv")

In [ ]:
all_studies_aa = pd.read_csv("/global/cfs/cdirs/kbase/KE-Catboost/HK/pfam_amino_acid_all_study_together.csv")
all_studies_aa = all_studies_aa.drop(['Unnamed: 0'], axis=1)
all_studies_aa['label_sensory'] = all_studies_aa.pfam_id+"-"+all_studies_aa.gene_oid+"-"+all_studies_aa.sensory_aa
all_studies_aa['biome'] = all_studies_aa['Ecosystem Category']+":"+all_studies_aa['Ecosystem Subtype']
all_studies_aa.to_csv("/global/cfs/cdirs/kbase/KE-Catboost/HK/pfam_amino_acid_all_study_togehter.csv", index='False')

#### chop out the sensory domain from each protein

In [ ]:
sliced = []#pd.DataFrame()
i=0
for aa in all_studies_aa.faa_sequence:
    # cut out query
    shortened = aa[all_studies_aa.query_start[i]:all_studies_aa.query_end[i]]
    sliced.append(shortened)
    i+=1
all_studies_aa['sensory_aa'] = sliced

#### just drop to non-duplicate amino acids to avoid more clustering time. Some gene-pfams have multiple pfam domains/multiple sensory_aa's. So the label must be gene_oid-pfam-sensory_aa to have all genes and all sensory aa in each gene cluster

In [ ]:
all_studies_aa['label_protein'] = all_studies_aa.pfam_id+"-"+all_studies_aa.gene_oid
all_studies_aa.label_protein.nunique()

In [ ]:
all_studies_aa['biome'].value_counts()

Chop out AA corresponding to sensory domain and save as a fasta file

## Make fasta file of sensory domains
* make 1 for each biome grouping and then add them all together to avoid crashing the kernel

In [ ]:
fasta = []
i=0
for aa in all_studies_aa.label_sensory:
    fasta.append(">%s\n%s" % (aa, all_studies_aa.sensory_aa[i]))
    i += 1
df = pd.Series(fasta).to_csv(index=False)
pd.Series(df.replace('"', '')).to_csv("/global/cfs/cdirs/kbase/KE-Catboost/HK/mmseqs/sensory_domains_eng.fasta", index=False)

# Processing the cluster file
## 1) Read in cluster file

In [7]:
clustered = pd.read_csv('/global/cfs/cdirs/kbase/KE-Catboost/HK/mmseqs/sensory_all_mode0_clu.tsv',
                        delimiter='\t', header=None, names=['cluster_base','label_sensory'])
clustered.head()

,cluster_base,label_sensory
0,pfam13188-CVPL005L_100016672-RIRNATIIKMFDQMSQA...,pfam13188-CVPL005L_100016672-RIRNATIIKMFDQMSQA...
1,pfam13188-CVPL005L_100016672-RIRNATIIKMFDQMSQA...,pfam13188-CVPL005L_100011619-IQMLNQIEQAIVITDQE...
2,pfam13188-CVPL005L_100016672-RIRNATIIKMFDQMSQA...,pfam13188-CVPL005W_10005012-RIRNATIIKMFDQMSQAI...
3,pfam13188-CVPL005L_100016672-RIRNATIIKMFDQMSQA...,pfam13188-CVPL010W_100011399-IQMLNQIEQAIVITDQE...
4,pfam13188-CVPL005L_100016672-RIRNATIIKMFDQMSQA...,pfam13188-CVPL010W_100046382-RIRNATIIKMFDQMSQA...


# pull out the 'bar' which is the cluster # for each sensory label

In [8]:
cluster_index = clustered['cluster_base'].drop_duplicates()
cluster_index = cluster_index.reset_index(drop=True).rename_axis('bar').reset_index()

In [9]:
clustered = clustered.merge(cluster_index, on='cluster_base', how='left')
clustered.head()

,cluster_base,label_sensory,bar
0,pfam13188-CVPL005L_100016672-RIRNATIIKMFDQMSQA...,pfam13188-CVPL005L_100016672-RIRNATIIKMFDQMSQA...,0
1,pfam13188-CVPL005L_100016672-RIRNATIIKMFDQMSQA...,pfam13188-CVPL005L_100011619-IQMLNQIEQAIVITDQE...,0
2,pfam13188-CVPL005L_100016672-RIRNATIIKMFDQMSQA...,pfam13188-CVPL005W_10005012-RIRNATIIKMFDQMSQAI...,0
3,pfam13188-CVPL005L_100016672-RIRNATIIKMFDQMSQA...,pfam13188-CVPL010W_100011399-IQMLNQIEQAIVITDQE...,0
4,pfam13188-CVPL005L_100016672-RIRNATIIKMFDQMSQA...,pfam13188-CVPL010W_100046382-RIRNATIIKMFDQMSQA...,0


In [ ]:
clustered['pfam_predicted'] = clustered['cluster_base'].str.split('-').str[0]
clustered['pfam'] = clustered['label_sensory'].str.split('-').str[0]
clustered['gene_predicted'] = clustered['cluster_base'].str.split('-').str[1]
clustered['gene_oid'] = clustered['label_sensory'].str.split('-').str[1]
clustered = clustered.drop(['cluster_base', 'label_sensory'],axis=1)
clustered.to_csv("/global/cfs/cdirs/kbase/KE-Catboost/HK/mode_0/clustered_mode_0_saved.csv")

# Catboost matrix prep only: 
* Select only clusters > 100 metagenomes

# For all matrix:
* calculate abundance using make_abundance.py script

In [ ]:
groupby_df = all_studies_aa_2.groupby('bar').filter(lambda x : len(x)>100)
groupby_df.to_csv("/global/cfs/cdirs/kbase/KE-Catboost/HK/clustered_all_over100_metagenomes.csv")
groupby_df = pd.read_csv('/global/cfs/cdirs/kbase/KE-Catboost/HK/clustered_all_over100_metagenomes.csv')
groupby_df = groupby_df.drop(['Unnamed: 0'], axis=1)
groupby_df.head()

groupby_df['is_cluster'] = True
groupby_df[['gene', 'bar', 'is_cluster']].to_csv('/global/cfs/cdirs/kbase/KE-Catboost/HK/clustered_100_for_df.csv')

groupby_df['is_cluster'] = True
groupby_df[['gene', 'bar', 'is_cluster']].to_csv('/global/cfs/cdirs/kbase/KE-Catboost/HK/clustered_100_for_df.csv')
groupby_df[['gene', 'bar', 'is_cluster']].head()

# Visualize the clusters graphically
* bring in the biome, and gold ID for all
* check for the mmseq2 the % pfam identity in the cluster

In [ ]:
clustered = pd.read_csv("/global/cfs/cdirs/kbase/KE-Catboost/HK/mode_0/clustered_mode_0_saved.csv")
clustered = clustered.drop('Unnamed: 0',axis=1).drop_duplicates()
cds = pd.read_csv("/global/cfs/cdirs/kbase/KE-Catboost/HK/IMG_dataset_Sean_Filtered.csv")
aa = pd.read_csv("/global/cfs/cdirs/kbase/KE-Catboost/HK/pfam_amino_acid_all_study_together.csv",
                 usecols = ['gene_oid','Ecosystem','Ecosystem Category','Ecosystem Subtype','GOLD Analysis Project ID'])
aa['biome'] = aa['Ecosystem']+":"+aa['Ecosystem Category']+":"+aa['Ecosystem Subtype']
aa = aa[['gene_oid','GOLD Analysis Project ID', 'biome']].drop_duplicates()
clustered = clustered.merge(aa, on='gene_oid', how='left')

In [ ]:
print('length:', len(clustered))
clustered.head()

In [ ]:
clustered.to_csv("/global/cfs/cdirs/kbase/KE-Catboost/HK/mode_0/mode_0_visualized_clusteres.csv")

### Count percent of pfam that are in the 'right' clusterclustered.pfam_predicted == clustered.pfam

In [ ]:
count_pfam_correct = (clustered.pfam_predicted == clustered.pfam).sum()
print('pfam correct: ',count_pfam_correct / len(clustered))

# Count of the sensory domains and proteins / biome

In [ ]:
clustered['GOLD Analysis Project ID'].nunique()

In [ ]:
aa_visualized = groupby_df[['sd_aa', 'biome']]
aa_visualized_2 = aa_visualized.drop_duplicates()
aa_visualized = aa_visualized.groupby(['sd_aa'])['biome'].nunique()
aa_visualized = pd.DataFrame(aa_visualized).reset_index()
aa_visualized = aa_visualized.groupby('biome')['sd_aa'].count().reset_index()
aa_visualized_2 = groupby_df.groupby(['sd_aa_predicted'])['biome'].nunique()
aa_visualized_2 = pd.DataFrame(aa_visualized_2).reset_index()
aa_visualized_2 = aa_visualized_2.groupby('biome')['sd_aa_predicted'].count().reset_index()

In [ ]:
aa_visualized['log(unique sensory domains)'] = np.log2(aa_visualized.sd_aa)
aa_visualized['log(cluster sensory domains)'] = np.log2(aa_visualized_2.sd_aa_predicted)

In [ ]:
aa = aa_visualized.merge(aa_visualized_2, on='biome', how='left')

In [ ]:
dfm = aa_visualized.drop('sd_aa',axis=1).melt('biome', var_name='cols', value_name='vals')

In [ ]:
alt.Chart(dfm, title="Count # Biomes Sensory Domain Found In").mark_circle().encode(
    x='biome',
    y='vals', tooltip=['biome'],
    color='cols').configure_range().properties(width=400,height=300)

# Count the number of unique biomes in each cluster

In [70]:
clustered_grouped = clustered.groupby('bar').filter(lambda x : len(x)>100)
clustered_grouped = clustered_grouped.groupby(['bar'])['biome'].nunique()
clustered_grouped = pd.DataFrame(clustered_grouped).reset_index()
clustered_grouped

,bar,biome
0,4,24
1,36,48
2,37,28
3,52,29
4,65,24
...,...,...
13497,113110,80
13498,113127,38
13499,113135,37
13500,113166,48


In [ ]:
clustered_grouped = clustered_grouped.groupby('biome')['bar'].count().reset_index()

In [ ]:
alt.Chart(clustered_grouped, title="Count of Unique Biomes / Cluster").mark_circle().encode(
    x='biome',
    y='bar', tooltip=['biome', 'bar'],
    color='bar').configure_range(
    category={'scheme':'dark2'}
).properties(width=400,height=300)

# Rare clusters only in a few biomes

In [85]:
one_two_biomes = clustered_grouped[clustered_grouped.biome==1]
one_two_biomes["one_two"] = True
one_two_biomes['biome_count'] = one_two_biomes['biome']
cl = clustered.groupby('bar').filter(lambda x : len(x)>100)
cl = cl[['bar', 'biome']].drop_duplicates()
groupby_df_one_two = cl.merge(one_two_biomes[['bar','one_two', 'biome_count']], how='left', on='bar')

groupby_df_one_two = groupby_df_one_two[groupby_df_one_two.one_two == True]

/tmp/ipykernel_63840/1952406927.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  one_two_biomes["one_two"] = True
/tmp/ipykernel_63840/1952406927.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  one_two_biomes['biome_count'] = one_two_biomes['biome']


### Which clusters < 3 biomes, what are those biomes?
#### They are not super spread out - only a few biomesgroupby_df_one_two

In [ ]:
groupby_df_one_two

In [83]:
#groupby_df_one_two['bar'] = np.log2(groupby_df_one_two['bar'])
alt.Chart(groupby_df_one_two, title="Clusters in < 3 Biomes").mark_circle(size=100).encode(
    x=alt.X('bar'),#scale=alt.Scale(domain=[10, 16])),
    y='biome', tooltip=['biome', 'bar'],
    color='biome').configure_range(
    category={'scheme':'tableau20'}
)#.configure_axis(
  #  labelFontSize=14,
   # titleFontSize=14).interactive()



alt.Chart(...)

#### The clusters for algae, insect, bird:large intestine, plant leaf?

#### 4352, 19268, 4490, 14283, 13260, 17741, 16247

### Which clusters == 1 biomes, what are those biomes?

In [86]:
alt.Chart(groupby_df_one_two, title="Clusters in 1 Biomes").mark_circle().encode(
    x='bar',
    y='biome', tooltip=['biome', 'bar'],
    color='biome').configure_range(
    category={'scheme':'tableau20'}
)

alt.Chart(...)

# Which genomes have a bad CDS count?

In [ ]:
cds = pd.read_csv("/global/cfs/cdirs/kbase/KE-Catboost/HK/IMG_dataset_Sean_Filtered.csv")

In [ ]:
cds['assembled.CDS.Count']

bin_labels_5 = ['<25k', '<50k', '<100k', '>100k']
cds['CDS_bins'] = pd.cut(cds['assembled.CDS.Count'],
                              bins=[-1, 25000, 50000, 100000, np.inf],
                              labels=bin_labels_5)
cds.head()

In [ ]:
cds = cds.groupby('biome').filter(lambda x : len(x) > 30)
cds_grouped = pd.DataFrame(cds[['biome','CDS_bins']].groupby(['biome','CDS_bins'])['CDS_bins'].count())
cds_grouped = cds_grouped.rename(columns={"CDS_bins": "count"}).reset_index()
cds_grouped = cds_grouped[cds_grouped['count'] > 0].reset_index(drop=True)
cds_grouped.head()

In [ ]:
cds_grouped = cds_grouped[cds_grouped.CDS_bins!='>100k']
alt.Chart(cds_grouped[cds_grouped['count']>20]).mark_bar().encode(
    y='biome',
    x='count',
    color='CDS_bins').configure_range(
    category={'scheme':'tableau20'}
)

In [ ]:
alt.Chart(cds_grouped[cds_grouped.CDS_bins=='<25k']).mark_arc(innerRadius=50).encode(
    theta=alt.Theta(field="count", type="quantitative"),
    color=alt.Color(field="biome", type="nominal"),
)